In [1]:
# Change directory to the root of the project
import os 
os.chdir('../../..')
print(f"Working directory: {os.getcwd()}")

Working directory: /Users/eohjelle/Documents/2025-mcts-playground/mcts-playground


This notebook takes a pretrained 5 layer resnet model, adds 3 layers to it and saves to file.

In [4]:
import torch
from core import Model
from experiments.connect_four.models.resnet import ResNet, ResNetInitParams

old_params = ResNetInitParams(in_channels=2, num_residual_blocks=5, channels=64, rows=6, cols=7, policy_head_dim=7) # For comparison
new_params = ResNetInitParams(in_channels=2, num_residual_blocks=8, channels=64, rows=6, cols=7, policy_head_dim=7)

In [5]:
full_old_model = Model.from_file(
    model_architecture=ResNet,
    path="experiments/connect_four/resnet_model.pt",
    device=torch.device('cpu')
)
full_new_model = Model(
    model_architecture=ResNet,
    init_params=new_params,
    device=torch.device('cpu')
)

In [7]:
# Try to load old model weights into larger model.

# copy what matches, ignore everything else
missing, unexpected = full_new_model.model.load_state_dict(
    full_old_model.model.state_dict(),
    strict=False          # ← key option
)

print("Weights copied for", len(full_old_model.model.state_dict()), "tensors")
print("Missing keys (new layers):", missing)        # should list block 5-7 params
print("Unexpected keys:", unexpected)               # usually empty here

Weights copied for 76 tensors
Missing keys (new layers): ['residual_layers.5.conv1.weight', 'residual_layers.5.bn1.weight', 'residual_layers.5.bn1.bias', 'residual_layers.5.bn1.running_mean', 'residual_layers.5.bn1.running_var', 'residual_layers.5.conv2.weight', 'residual_layers.5.bn2.weight', 'residual_layers.5.bn2.bias', 'residual_layers.5.bn2.running_mean', 'residual_layers.5.bn2.running_var', 'residual_layers.6.conv1.weight', 'residual_layers.6.bn1.weight', 'residual_layers.6.bn1.bias', 'residual_layers.6.bn1.running_mean', 'residual_layers.6.bn1.running_var', 'residual_layers.6.conv2.weight', 'residual_layers.6.bn2.weight', 'residual_layers.6.bn2.bias', 'residual_layers.6.bn2.running_mean', 'residual_layers.6.bn2.running_var', 'residual_layers.7.conv1.weight', 'residual_layers.7.bn1.weight', 'residual_layers.7.bn1.bias', 'residual_layers.7.bn1.running_mean', 'residual_layers.7.bn1.running_var', 'residual_layers.7.conv2.weight', 'residual_layers.7.bn2.weight', 'residual_layers.7.bn

In [8]:
full_new_model.save_to_file("experiments/connect_four/resnet_model_large.pt")